In [1]:
import os
os.getcwd()
os.chdir("../..")
os.getcwd()

'/home/caleb/Cornell-Conversational-Analysis-Toolkit'

In [2]:
import convokit
import pickle
import pandas as pd


In [3]:
from convokit import Corpus, Conversation, Utterance

In [4]:
fake_news_dir = '/sauna/fake-news'
politics_dir = '/sauna/reddit_201810_raw/corpus/pokemontrades_banlist~-~politics/politics'
donald_dir = '/sauna/reddit_201810_raw/corpus/TheTwoBeerQueers~-~The_Donald/The_Donald/'

In [5]:
fake_news_urls = {'breitbart.com', 'dcgazette.com', "dailymail.co.uk", 
                  "lewrockwell.com", "newswars.com", "gellerreport.com",
                  "dcclothesline.com", "thegatewaypundit.com", "trueactivist.com"
                 }

In [6]:
real_news_sites = ['BBC', 'Business Insider', 'Buzzfeed', 'CBS News', 'CNN', 'Daily Beast', 
                   'FT Westminster Blog', 'FiveThirtyEight', 'Fortune', 'Mercury News', 'NPR',
                   'National Review', 'New Yorker', 'Newsweek', 'PBS', 'Politico', 'Real Clear Politics', 'Reuters',
                   'Slate', 'The American Conservative', 'The Denver Post', 'The Guardian', 'The Hill', 'The New York Times',
                   'The Verge', 'USA Today', 'Vox', 'WSJ Washington Wire', 'Washington Monthly', 'Washington Post']

In [7]:
real_news_urls = {'bbc.com', 'businessinsider.com', 'buzzfeed.com', 'cbsnews.com', 'cnn.com', 'thedailybeast.com',
                  'ft.com', 'fivethirtyeight.com', 'fortune.com', 'mercurynews.com', 'npr.org', 'nationalreview.com',
                  'newyorker.com', 'newsweek.com', 'pbs.org', 'politico.com', 'realclearpol', 'reuters.com',
                  'slate.com', 'theamericanconservative.com', 'denverpost.com', 'theguardian.com', 'thehill.com',
                  'nytimes.com', 'theverge.com', 'usatoday.com', 'vox.com', 'wsj.com', 'washingtonmonthly.com', 
                  'washingtonpost.com'
                 }

In [8]:
politics_corpus = Corpus(filename=politics_dir)
# donald_corpus = Corpus(filename=os.path.join(fake_news_dir, 'donald_corpus_valid_convos'))

In [ ]:
# Start time: 01 Oct 2017
# End time: 01 Oct 2018
start_time = 1475280000
end_time = 1538352000

In [ ]:
from collections import defaultdict
real_news_convos = defaultdict(set)
fake_news_convos = defaultdict(set)
for convo in politics_corpus.iter_conversations():
    if start_time <= convo.meta['timestamp'] <= end_time:
        if convo.meta['domain'] in real_news_urls:
            real_news_convos[convo.meta['domain']].add(convo.id)
        elif convo.meta['domain'] in fake_news_urls:
            fake_news_convos[convo.meta['domain']].add(convo.id)

In [11]:
for k, v in real_news_convos.items():
    print("{}: {}".format(k, len(v)))
    
print("Total posts:", sum([len(v) for v in real_news_convos.values()]))

politico.com: 21395
thehill.com: 55517
washingtonpost.com: 78430
cnn.com: 28123
nytimes.com: 36655
buzzfeed.com: 6202
businessinsider.com: 9968
npr.org: 7665
reuters.com: 16612
usatoday.com: 9017
ft.com: 582
theguardian.com: 14605
fivethirtyeight.com: 1678
nationalreview.com: 2222
thedailybeast.com: 10615
pbs.org: 1410
vox.com: 9811
newyorker.com: 2534
bbc.com: 5475
cbsnews.com: 6077
fortune.com: 1453
slate.com: 5701
mercurynews.com: 599
wsj.com: 5005
theverge.com: 981
newsweek.com: 9283
theamericanconservative.com: 282
denverpost.com: 678
washingtonmonthly.com: 502
Total posts: 349077


In [12]:
for k, v in fake_news_convos.items():
    print("{}: {}".format(k, len(v)))
    
print("Total posts:", sum([len(v) for v in fake_news_convos.values()]))

breitbart.com: 5861
dailymail.co.uk: 3071
thegatewaypundit.com: 683
lewrockwell.com: 30
trueactivist.com: 19
dcclothesline.com: 16
gellerreport.com: 4
newswars.com: 2
Total posts: 9686


In [13]:
from itertools import chain
real_news_convos_ids = list(chain.from_iterable(real_news_convos.values()))
fake_news_convos_ids = list(chain.from_iterable(fake_news_convos.values()))

In [14]:
valid_convos_ids = set(real_news_convos_ids + fake_news_convos_ids)

In [15]:

politics_corpus.print_summary_stats()

Number of Users: 1772618
Number of Utterances: 89059584
Number of Conversations: 3243950


In [16]:
politics_corpus.filter_conversations_by(func=lambda convo: convo.id in valid_convos_ids)

In [17]:
politics_corpus.print_summary_stats()

Number of Users: 599761
Number of Utterances: 20075242
Number of Conversations: 358763


In [18]:
politics_corpus.dump('politics_corpus_valid_convos_2year', base_path=fake_news_dir)

KeyboardInterrupt: 

In [19]:
thread_pfxs = politics_corpus.utterance_threads(prefix_len=8, include_root=False)
convos_with_valid_len = set()
convo_to_thread = defaultdict(list)
for thread_id, utts in thread_pfxs.items():
    if len(utts) >= 8:
        convos_with_valid_len.add(utts[next(iter(utts))].root)
        convo_to_thread[utts[next(iter(utts))].root].append(thread_id)

In [20]:
from random import choice

for convo, threads in convo_to_thread.items():
    convo_to_thread[convo] = choice(threads)

In [21]:
thread_to_convo = {v: k for k, v in convo_to_thread.items() if len(v) > 0}

In [22]:
real_news_thread_ids = [convo_to_thread[convo_id] for convo_id in real_news_convos_ids if len(convo_to_thread[convo_id]) > 0]
fake_news_thread_ids = [convo_to_thread[convo_id] for convo_id in fake_news_convos_ids if len(convo_to_thread[convo_id]) > 0]



In [23]:
len(real_news_thread_ids)

86760

In [24]:
len(fake_news_thread_ids)

3033

In [25]:
from random import sample
real_news_thread_sample = sample(real_news_thread_ids, len(fake_news_thread_ids))

In [26]:
real_news_convos_sample = [thread_to_convo[thread_id] for thread_id in real_news_thread_sample]

In [27]:
fake_news_long_convos = [thread_to_convo[thread_id] for thread_id in fake_news_thread_ids]

In [28]:
paired_convos = set(real_news_convos_sample).union(fake_news_long_convos)

In [29]:
balanced_threads = fake_news_thread_ids + real_news_thread_sample

In [30]:
len(set(balanced_threads))

6066

In [31]:
len(paired_convos)

6066

In [32]:
politics_corpus.filter_conversations_by(func=lambda convo: convo.id in paired_convos)

In [33]:
politics_corpus.dump('politics_corpus_paired_2year', base_path=fake_news_dir)

In [ ]:
# donald_corpus = Corpus(filename=os.path.join(fake_news_dir, 'donald_corpus_paired'))

In [34]:
politics_corpus.print_summary_stats()

Number of Users: 121768
Number of Utterances: 812227
Number of Conversations: 6066


In [35]:
hc = convokit.HyperConvo(min_thread_len=8, prefix_len=8, include_root=False)

In [36]:
motif_feats = hc.retrieve_motif_counts(politics_corpus)

In [37]:
balanced_threads = set(balanced_threads)

In [38]:
motif_feats = {k: v for k, v in motif_feats.items() if k in balanced_threads}

In [39]:
motif_counts_df = pd.DataFrame.from_dict(motif_feats, orient='index')
motif_feat_names = list(motif_counts_df.columns)

In [40]:
hyperconv_feats = hc.retrieve_feats(politics_corpus)
hyperconv_feats = {k: v for k, v in hyperconv_feats.items() if k in balanced_threads}

/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:182: RuntimeWarning: invalid value encountered in long_scalars
  "norm.max": lambda l: np.max(l) / np.sum(l),
/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:187: RuntimeWarning: invalid value encountered in long_scalars
  if len(l) > 1 else np.nan,
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2614: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=0)
/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:194: Runtim

In [41]:
hyperconv_df = pd.DataFrame.from_dict(hyperconv_feats, orient='index')
hyperconv_feat_names = list(hyperconv_df.columns)

In [42]:
path_feats = hc.retrieve_motif_pathway_stats(politics_corpus)
path_feats = {k: v for k, v in path_feats.items() if k in balanced_threads}

In [43]:
path_stats_df = pd.DataFrame.from_dict(path_feats, orient='index')
columns = ['PATH-'+', '.join(filter(lambda x: type(x) == str, col)).strip() for col in path_stats_df.columns.values]
path_stats_df.columns = columns
path_feat_names = list(path_stats_df.columns)

In [44]:
feats_df = pd.concat([hyperconv_df, path_stats_df, motif_counts_df], axis=1)

In [45]:
feats_df['volume'] = [len(set(utt.user.name for utt in thread_pfxs[thread_id].values())) for thread_id in feats_df.index]

In [46]:
fake_news_thread_ids = set(fake_news_thread_ids)
y = [int(thread_id in fake_news_thread_ids) for thread_id in list(feats_df.index)]

In [47]:
import numpy as np
y = np.array(y)

In [52]:
feats_df['y'] = y

In [53]:
feats_df.shape

(6066, 262)

In [54]:
feats_df_no_na = feats_df.dropna()

In [55]:
len(feats_df_no_na)

5897

In [56]:
sum(feats_df_no_na['y'])

2918

In [58]:
def print_extreme_coefs(clf, feature_names, num_features: int = 5):
    coefs = clf.named_steps['logreg'].coef_[0].tolist()

    assert len(feature_names) == len(coefs)

    feats_coefs = sorted(list(zip(feature_names, coefs)), key=lambda x: x[1], reverse=True)

    print()
    print("TOP {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[:num_features]:
        print("{}: {:.3f}".format(ft, coef))
    print()
    print("BOTTOM {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[-num_features:]:
        print("{}: {:.3f}".format(ft, coef))
    print()

In [60]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


for feature_set, name in [(hyperconv_feat_names, "hyperconvo"),
                        (["volume"], "volume"),
                        (hyperconv_feat_names + ["volume"], "hyperconv-volume"),
                        (motif_feat_names, "motifs"),
                        (motif_feat_names + ["volume"], "motifs-volume"),
                        (path_feat_names, "motifpaths"),
                        (path_feat_names + ["volume"], "motifpaths-volume"),
                        (hyperconv_feat_names + motif_feat_names, "hyperconv-motif"),
                        (hyperconv_feat_names + path_feat_names, "hyperconv-paths"),
                        (hyperconv_feat_names + motif_feat_names + path_feat_names, "hyperconvo-motifall"),
                       ]:
    clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])      

    X = np.array(feats_df_no_na[feature_set])
    y = np.array(feats_df_no_na['y'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    test_score = clf.score(X_test, y_test)
    print("- {}, train: {:.4f}, test: {:.4f}".format(name, train_score, test_score))
#     print_extreme_coefs(clf, feature_set, num_features = 5)
    

- hyperconvo, train: 0.6084, test: 0.6008
- volume, train: 0.5391, test: 0.5288
- hyperconv-volume, train: 0.6076, test: 0.6000
- motifs, train: 0.5595, test: 0.5610
- motifs-volume, train: 0.5629, test: 0.5576
- motifpaths, train: 0.5652, test: 0.5653
- motifpaths-volume, train: 0.5699, test: 0.5627
- hyperconv-motif, train: 0.6129, test: 0.5975
- hyperconv-paths, train: 0.6127, test: 0.5856
- hyperconvo-motifall, train: 0.6127, test: 0.5864


In [61]:
thread_pfxs = politics_corpus.utterance_threads(prefix_len=8, include_root=False)

## BoW prediction

In [62]:
thread_pfx_texts = {k: " ".join([utt.text for utt in v.values()]) for k, v in thread_pfxs.items() if k in balanced_threads}

In [63]:
y = [int(thread_id in real_news_thread_sample) for thread_id in list(feats_df.index)]

In [64]:
train_ids, test_ids, y_train, y_test = train_test_split(list(balanced_threads), y, test_size=0.2, random_state=42)

In [65]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0.05, max_df=0.7, ngram_range=(1,3))

In [66]:
X_train = cv.fit_transform([thread_pfx_texts[train_id] for train_id in train_ids]).toarray()

In [67]:
X_test = cv.transform([thread_pfx_texts[test_id] for test_id in test_ids]).toarray()

In [68]:
clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])      

clf.fit(X_train, y_train)
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)
print("- {}, train: {:.4f}, test: {:.4f}".format("BoW", train_score, test_score))


- BoW, train: 0.6358, test: 0.5049


In [69]:
print_extreme_coefs(clf, cv.get_feature_names(), num_features = 10)


TOP 10 FEATURES
http www: 0.280
https www: 0.271
fake: 0.141
get: 0.137
will: 0.132
all: 0.131
have been: 0.130
kind of: 0.119
the gop: 0.114
than: 0.106

BOTTOM 10 FEATURES
gop: -0.105
information: -0.107
to get: -0.120
out: -0.122
would: -0.123
kind: -0.127
http: -0.132
https: -0.136
been: -0.215
www: -0.372

